# GRADTDA5622 - Big Data Computing Foundations 2
## Homework 2: Manipulating Data with SQL
- Semester: Spring 2023
- Instructor: Tom Bihari
- Section: N/A
- Student Name: Able Baker **(fill in)**
- Student Email: baker.12345@osu.edu **(fill in)**
- Student ID: 123456789 **(fill in)**
***

***
# Section: Overview
***

**The Objectives of This Assignment are:**
1. To gain experience using SQL to manipulate data to solve problems and answer questions.

**Instructions:**
- **Follow the instructions** in each section.
- **Fill in** the **Conclusions** section.

***
# Section: Setup
- Add any needed imports, helper functions, etc., here.
- **I added all of these helper functions... Tom Bihari**
***

In [ ]:
import os
import pandas as pd
import sqlite3 as sq

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

data_directory = "../shared_Sp23/"  # Set this to the appropriate directory for your setup.

In [ ]:
# Connect to a SQLite database (will be created if it doesn't exist).
def connect_db(file_name):
    if not os.path.isfile(file_name):
        print('Database file does not exist and will be created.')
    try:
        conn = sq.connect(file_name)
        print("Database connection established.")
        return conn
    except Error:
        print(Error)
        return None

In [ ]:
# Create a cursor to use to access the data.
def get_cursor(connection):
    try:
        cur = connection.cursor()
        print('Cursor created.')
        return cur
    except Error:
        print(Error)
        return None

In [ ]:
# Close the connection to the database.
def close_db(connection, cursor):
    connection.commit()
    cursor.close()
    connection.close()

In [ ]:
# Create a SQLite table by reading a CSV file.  Uses Pandas.
def create_table_from_csv(connection, file_name, table_name):
    pdf = pd.read_csv(file_name)
    create_table_from_dataframe(connection, pdf, table_name)

In [ ]:
# Create a SQLite table by reading a worksheet in an excel file.  Uses Pandas.
def create_table_from_excel(connection, file_name, sheet_name, table_name):
    pdf = pd.read_excel(file_name, sheet_name)
    create_table_from_dataframe(connection, pdf, table_name)

In [ ]:
# Create a SQLite table from a Pandas DataFrame.
def create_table_from_dataframe(connection, pdf, table_name):
    pdf.to_sql(table_name, connection, if_exists='replace', index = False)

In [ ]:
# Create a Pandas DataFrame from a SQLite cursor.
def create_dataframe_from_cursor(cursor):
    cols = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(data = cursor.fetchall(), columns = cols)

In [ ]:
# Create a Pandas DataFrame from a SQLite table (or view).
#   Or could do: df = pd.read_sql_query("SELECT * FROM table_name", cnx)
#   See: https://stackoverflow.com/questions/36028759/how-to-open-and-convert-sqlite-database-to-pandas-dataframe
def create_dataframe_from_table(connection, table_name):
    cursor = run_query(connection,'SELECT * FROM "' + table_name + '"')
    cols = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(data = cursor.fetchall(), columns = cols)

In [ ]:
# Run a SQLite query and return the result.
def run_query(conn_or_cur, query_string):
    try:
        result = conn_or_cur.execute(query_string)
    except:
        print("Query error")
        result = None
    return result

In [ ]:
# Replace text \n and \t with actual newline and tab characters.
def clean_string(strng):
    return str(strng).replace('\\n', '\n').replace('\\t', '\t')

In [ ]:
# Print query results up to a specified max number of records).
def print_result(cursor, max_num_records):
    print("------------------------------------------")
    df = create_dataframe_from_cursor(cursor)
    #print(df.head(max_num_records))
    display(df.head(max_num_records))
    print("------------------------------------------")

In [ ]:
def print_all_table_and_view_names(conn_or_cur):
    result = run_query(conn_or_cur,'SELECT name from sqlite_master where type= "table"')
    print("\nALL TABLE NAMES: ",result.fetchall())
    result = run_query(conn_or_cur,'SELECT name from sqlite_master where type= "view"')
    print("ALL VIEW NAMES:  ",result.fetchall())

In [ ]:
def print_schema(conn_or_cur, table_or_view_name):
    #https://www.sqlite.com/schematab.html
    # Table "sqlite_schema" is correct, but "sqlite_master" is used for backward compatibility.)
    result = run_query(conn_or_cur,'SELECT sql from sqlite_master where name= "' + table_or_view_name + '"')
    print("\nSCHEMA: " + table_or_view_name)
    print(clean_string(result.fetchall()))

In [ ]:
def print_table_or_view(conn_or_cur, table_name, max_num_records):
    print("\nTABLE/VIEW: " + table_name)
    result = run_query(conn_or_cur,'SELECT * FROM "' + table_name + '"')
    print_result(result, max_num_records)

---
# Set up a Database
- Create a blank database.
- Create three tables from the **state-abbrevs.csv**, **state-areas.csv**, and **state-populations.csv** files in the shared data folder.
- **I have done these steps for you**, from the SQL Examples notebook.
---

## Connect to a blank database

In [ ]:
# Connect to the database and get a cursor to access the data.
#conn = connect(r'SQLiteUSStatesData.db')  # a file database
conn = connect_db(r':memory:')             # an in-memory database
cur = get_cursor(conn)  # not needed for these examples, but kept for compatibility

## Create some tables by reading data from CSV files

In [ ]:
create_table_from_csv(conn, data_directory + 'state-abbrevs.csv', 'abbrevs_table')
print_schema(conn,"abbrevs_table")
print_table_or_view(conn, "abbrevs_table", 3) # Show a few example records

In [ ]:
create_table_from_csv(conn, data_directory + 'state-areas.csv', 'areas_table')
#print_schema(conn,"areas_table")
print_table_or_view(conn, "areas_table", 3) # Show a few example records

In [ ]:
create_table_from_csv(conn, data_directory + 'state-populations.csv', 'populations_table')
#print_schema(conn,"populations_table")
print_table_or_view(conn, "populations_table", 3) # Show a few example records

In [ ]:
print_all_table_and_view_names(conn)

---
# Task 1: Create a Table of Summarized State Populations
- Get the min, average, and max total and under18 populations for each of the states over all years.
- Call the table **summarized_state_populations**.
- The table should have the following fields:
  - state, ages, min_pop, avg_pop, max_pop
  - It should be sorted by **state** (from A to Z), and within each state, by **under18** age range first, then **total** age range. 
---

In [ ]:
# Fill in the blanks
run_query(conn,'DROP TABLE IF EXISTS summarized_state_populations') # Delete the table if it already exists

run_query(conn,'''
    CREATE TABLE IF NOT EXISTS summarized_state_populations
    AS SELECT state, ages, ...
    FROM populations_table
    GROUP BY ...
    ORDER BY ...
    ''')

print_table_or_view(conn, "summarized_state_populations", 3)

---
# Task 2: Print the Top 3 States with the Largest Normalized Population Swings
- Consider the difference between max and min "total" populations for each state.
- Normalize by dividing by the average population for the state.
- You can use the table from the previous query.
---

In [ ]:
# Fill in the blanks

result = run_query(conn,'''
    SELECT state, min_pop, max_pop, ...
    FROM summarized_state_populations
    WHERE ages = "total"
    ORDER BY ...
    ''')

print_result(result, 3)

---
# Task 3: Create a Table of Population Densities
- Follow each of the individual steps below.  There likely are shorter ways to do this task, but this way the steps are explicit.
- I have given the shells of some of the queries, so you can "fill in the blanks".
- I have included the expected outputs, so you can see if yours matches.
- Name the table **population_densities_table**
- The table should have the following fields:
  - state_abbrev
  - state_name
  - area
  - year
  - population_total
  - population_under_18
  - population_18_and_over
  - density_total
  - density_under_18
  - density_18_and_over
---

## Create a "pop_and_name_table"
- Join the populations_table and abbrevs_table.
- Table should have fields: state_abbrev, state_name, ages, year, population.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS pop_and_name_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS pop_and_name_table
    AS SELECT populations_table.state AS state_abbrev, abbrevs_table.state AS state_name, ...
    FROM (populations_table LEFT OUTER JOIN ...)
    ''')

print_table_or_view(conn, "pop_and_name_table", 3) # Show a few records

## Create a "pop_name_area_density_table"
- Join the pop_and_name_table and areas_table.
- Table should have fields: state_abbrev, state_name, ages, year, population, area, density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS pop_name_area_density_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS pop_name_area_density_table
    AS SELECT pop_and_name_table.*, areas_table.area, ...
    FROM (pop_and_name_table LEFT OUTER JOIN areas_table ...)
    ORDER BY year ASC, state_abbrev ASC, ages ASC
    ''')

print_table_or_view(conn, "pop_name_area_density_table", 3) # Show a few records

## Create an "under18_table"
- Select only the "under18" ages records from the pop_name_area_density_table.
- Table should have fields: state_abbrev, state_name, year, population, area, density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS under18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS under18_table
    AS SELECT state_abbrev, state_name, year, ...
    FROM pop_name_area_density_table
    WHERE ages = ...
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "under18_table", 3) # Show a few records

## Create a "total_table"
- Select only the "total" ages records from the pop_name_area_density_table.
- Table should have fields: state_abbrev, state_name, year, population, area, density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS total_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_table
    AS SELECT state_abbrev, ...
    FROM pop_name_area_density_table
    WHERE ages = "total"
    ORDER BY ...
    ''')

print_table_or_view(conn, "total_table", 3) # Show a few records

## Create a "total_and_under18_table"
- Join the under18_table and total_table.
- Table should have fields: state_abbrev, state_name, year, area, total_pop, total_density, under18_pop, under18_density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS total_and_under18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_and_under18_table
    AS SELECT total_table.state_abbrev, total_table.state_name, total_table.year, total_table.area, 
        total_table.population as total_pop, total_table.density as total_density, under18_table.population as under18_pop, under18_table.density as ...
    FROM (total_table LEFT OUTER JOIN under18_table ON
        (total_table.state_abbrev = under18_table.state_abbrev AND total_table.year = ...))
    ''')

print_table_or_view(conn, "total_and_under18_table", 3) # Show a few records

## Create a "total_under18_over18_table"
- Start with the total_and_under18_table and calculate columns for the over18_pop, over18_density.
- Actually, we mean "18 and over", not "over 18".
- Table should have fields: state_abbrev, state_name, year, area, total_pop, total_density, under18_pop, under18_density, over18_pop, over18_density.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS total_under18_over18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_under18_over18_table
    AS SELECT *, (total_pop - under18_pop) as over18_pop, (...) as over18_density
    FROM total_and_under18_table
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "total_under18_over18_table", 3) # Show a few records

## Create the final population_densities_table"
- Start with the total_under18_over18_table.
- Rename and reorder the columns to match the original request.
- Sort by year and state abbreviation, ascending.
- Table should have fields: state_abbrev, state_name, area, year, 
  - population_total, population_under_18, population_18_and_over, density_total, density_under_18, density_18_and_over.

In [ ]:
run_query(conn,'DROP TABLE IF EXISTS population_densities_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS population_densities_table
    AS SELECT state_abbrev, state_name, area, year, total_pop AS population_total, ...,
        total_density AS density_total, under18_density as density_under_18, over18_density as density_18_and_over
    FROM total_under18_over18_table
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "population_densities_table", 10) # Show a few records

---
# Cleanup and Close the Current Database
---

In [ ]:
close_db(conn, cur)

---
# Task 4: Process Datasets with SQL and Answer Questions - From Scratch
In this Task, you will load several datasets into a new database.  You then will process the data and answer the following questions.  The code in Tasks 1-3 above can be used as examples.

The data is in an Excel file: 'MiscWikipediaCountryData.xls'.  You may open the file in Excel and look at it if you like.  The file contains four worksheets: GovernmentData, InternetData, IncomeEqualityData, and IncomePerCapitaData.

**Context and Hints**
- We are interested only in UN_Member countries.
- We don't care about the Year information.  We will assume that the data is relevant for the current year.
- Each dataset contains some of the information needed to answer the questions, so you will have to combine the information.  Consider building a single table "common_country_table" with columns:
  - **Country, Constitutional_Form, Head_Of_State, Subregion, Region, Population, Rich_Poor_Income_Ratio (use the CIA_RP10 column), Per_Capita_Income (use the IMF_Estimate column)**
  - Consider using JOIN operations.
- There is an issue with the data (as is usually the case...): Each dataset has a column that contains the country's name.  However, the datasets may contain different subsets of countries, and the country names may be spelled differently.  What should you do?  Here are some options:
  1. Treat one of the datasets (e.g., GovernmentData) as the primary dataset, and join the other datasets' data to the primary dataset IF you find a match; otherwise, join null information.
      - Pros: Gives a consistent set of countries.  Cons: Some fields will be null.
  2. Treat all of the datasets as "equal" and join information only if you find matching country names in all datasets.
      - Pros: No null fields, unless they were null in the original datasets.  Cons: A row will appear only if the country name appears in ALL datasets, so some countries will be missing!
  3. Do a deeper analysis and try to fix as many country name mismatches as possible.  Then join them.
      - Pros: May results in cleaner / more complete final product.  Cons: Difficult and probably manual work.
  4. Create a combined table of all of the country names across all of the datasets, and join all four datasets to that table.
      - Pros: Result will contain all information from all datasets.  Cons: A single real country may appear in multiple rows with different country names.

For simplicity, let's go with option 2.  What kind of JOIN accomplishes this?

To start things off, the initial setup has been given to you below.  Beneath that, you can fill in your own code.

---

## Connect to a blank database

In [ ]:
# Connect to the database and get a cursor to access the data.
conn = connect_db(r':memory:')             # an in-memory database
cur = get_cursor(conn)  # not needed for these examples, but kept for compatibility

## Create four tables by reading data from the Excel file
- Review the Excel file iself to see the source and metadata for the datasets.
- Both .xls and .xlsx versions of the Excel file are provided in the data directory.  Some Python installations may not recognize the .xlsx version, so use what works for you.
- Create four tables: **govt_table**, **internet_table**, **income_eq_table**, **income_perc_table**. The govt_table is already done for you.

In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'GovernmentData', 'govt_table')
print_schema(conn,"govt_table")
print_table_or_view(conn, "govt_table", 3) # Show a few example records

In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'InternetData', 'internet_table')
print_schema(conn,"internet_table")
print_table_or_view(conn, "internet_table", 3) # Show a few example records

In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'IncomeEqualityData', 'income_eq_table')
print_schema(conn,"income_eq_table")
print_table_or_view(conn, "income_eq_table", 3) # Show a few example records

In [ ]:
create_table_from_excel(conn, data_directory + 'MiscWikipediaCountryData.xls', 'IncomePerCapitaData', 'income_perc_table')
print_schema(conn,"income_perc_table")
print_table_or_view(conn, "income_perc_table", 3) # Show a few example records

---
## Your Code
In the area below, write the SQL code to answer the following questions.  You may add / modify / delete cells as you wish.  Make sure you include comments that describe:
- What you are trying to do, and why.
- How you are doing it (comments in code, if appropriate).
- Your results or findings, if any.

**Answer These Questions:**
1. What is the Population of each Region?
2. Which Constitutional_Form has the highest Population, for counties that have reported their Population?
3. Which Constitutional_Form has the highest average Per_Capita_Income, for counties that have reported their Per_Capita_Income?
4. Which Country has the lowest average Rich_Poor_Income_Ratio, for counties that have reported their Rich_Poor_Income_Ratio?
5. What is the [Region, Subregion, Country, Population] of each of the 10 most populous countries in our dataset?  What is unrealistic about this answer?
6. What are the top 10 Countries in terms of total income (consider Population * Per_Capita_Income)?
---

---
# Cleanup and Close the Current Database
---

In [ ]:
close_db(conn, cur)

***
# Section: Conclusions
- What did you learn from this exercise?
***

In [ ]:
# Fill this in.